In [ ]:
# 기본 도구 가져오기
import pickle
from konlpy.tag import Okt
okt = Okt()

def okt_tokenizer(text):
  tokens = okt.morphs(text)
  return tokens

In [ ]:
#학습된 모델 가져오기, tfidf, 영화평점, 약 학습,평가요 총
# 약 20만개 데이터를 기준으로 학습된 모델 불러오기.
tfidf_model_save_path = "./tfidf_model.pkl"
with open(tfidf_model_save_path, "rb") as file:
    tfidf = pickle.load(file)

print("✅ TF-IDF 모델 불러오기 완료")

In [ ]:

model_save_path = "./SA_lr_best.pkl"
with open(model_save_path, "rb") as file:
    SA_lr_best = pickle.load(file)

print("✅ 모델 불러오기 완료")


In [ ]:
# 학습된 모델을 평가 했더니, 정확도 약 85% 정도 일치
# 긍정, 부정을 분석 일치 여부.
# 임의로, 특정의 문장을 , 이 모델에 입력시키면, 그 결과가, 긍정?, 부정? 판단을 해줌.
# 정확도 약85% 이지만, 생각보다 성능이 많이 떨어짐. 참고하고
st = input("감성 분석하기위한 문장을 입력 해주세요: ")


In [ ]:
import re

st = re.compile(r"[ㄱ - | 가-힣]+").findall(st)
print(st)
st = [" ".join(st)]
print(st)


In [ ]:
# 해당 문장을 분석하기 좋게 벡터화 과정.
st_tfidf = tfidf.transform(st)

In [ ]:
# 만들어둔 모델을 이용해서, 예측값 구하기(평가)
st_predict = SA_lr_best.predict(st_tfidf)

In [ ]:
# 긍정 인지, 부정인지 판별하기.
if(st_predict == 0):
  print(st," -> 부정")
else:
  print(st," -> 긍정")

In [ ]:
# 코로나 뉴스 json -> 메모리에 불러오기.
import json
file_name = '트럼프_naver_news'
with open("./"+file_name+'.json',encoding='utf-8') as j_f:
  data = json.load(j_f)
print(data)

In [ ]:
# 해당 기사의 제목, 내용에 대해서,
# 기본적인 전처리 작업.
# 제목, 내용을 담을 임시 리스트 만들기.
data_title = []
data_description = []

for item in data:
  data_title.append(item["title"])
  data_description.append(item["description"])

# data_title
data_description

In [ ]:
# 리스트 -> 데이터 프레임 (표) 형태로 변환
import pandas as pd
data_df = pd.DataFrame({"title":data_title, "description" : data_description})
data_df.head()

In [ ]:
# 전처리, 1)제목 2)내용 한글만 남기기 작업.
# re (regex 정규식) 이용해서, 한글만 남기기.
import re
# 제목에서 한글만 추출
data_df["title"]= data_df["title"].apply(lambda x: re.sub(r'[^ ㄱ - | 가-힣]+'," ",x))
# 내용에서 한글만 추출
data_df["description"]= data_df["description"].apply(lambda x: re.sub(r'[^ ㄱ - | 가-힣]+'," ",x))
data_df.head()


In [ ]:
# 벡터화 작업
data_title_tfidf = tfidf.transform(data_df["title"])

In [ ]:
data_title_predict = SA_lr_best.predict(data_title_tfidf)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# 예측값을 새로운 컬럼으로 사용하기.
data_df["title_label"] = data_title_predict
# description , 같은 작업 반복
# 벡터화 작업
data_description_tfidf = tfidf.transform(data_df["description"])
# 학습이 된 감성 분석 모델을 이용해서, 해당 제목으로, 긍정, 부정인지 평가하기.
data_description_predict = SA_lr_best.predict(data_description_tfidf)
# 예측값을 새로운 컬럼으로 사용하기.
data_df["description_label"] = data_description_predict
data_df.head()
# 파일 csv 로 옮기기
data_df.to_csv("./" + file_name + ".csv", encoding="utf-8")
# 간단히, 해당 제목, 내용이 , 긍정, 부정의 글의 갯수가 몇개인지 파악해보기.
print(data_df["title_label"].value_counts())
print(data_df["description_label"].value_counts())
# 제목, 내용 분리 작업.
columns_name = ['title', 'title_label', 'description', 'description_label']
NEG_data_df = pd.DataFrame(columns=columns_name)
POS_data_df = pd.DataFrame(columns=columns_name)

for i, data in data_df.iterrows():
    title = data["title"]
    description = data["description"]
    t_label = data["title_label"]
    d_label = data["description_label"]

    if d_label == 0:  # 부정 감성 샘플만 추출
        # NEG_data_df = NEG_data_df.append(pd.DataFrame([[title, t_label, description, d_label]],columns=columns_name),ignore_index=True)
        new_data_df = pd.DataFrame([[title, t_label, description, d_label]], columns=columns_name)
        NEG_data_df = pd.concat([NEG_data_df, new_data_df], ignore_index=True)
    else:  # 긍정 감성 샘플만 추출
        # POS_data_df = POS_data_df.append(pd.DataFrame([[title, t_label, description, d_label]],columns=columns_name),ignore_index=True)
        new_data_df = pd.DataFrame([[title, t_label, description, d_label]], columns=columns_name)
        POS_data_df = pd.concat([POS_data_df, new_data_df], ignore_index=True)
# 파일에 저장.
NEG_data_df.to_csv('./' + file_name + '_NES.csv', encoding='utf-8')
POS_data_df.to_csv('./' + file_name + '_POS.csv', encoding='utf-8')
len(NEG_data_df), len(POS_data_df)
POS_description = POS_data_df['description']
POS_description_noun_tk = []

for d in POS_description:
    POS_description_noun_tk.append(okt.nouns(d))  #형태소가 명사인 것만 추출
print(POS_description_noun_tk)  #작업 확인용 출력
POS_description_noun_join = []

for d in POS_description_noun_tk:
    d2 = [w for w in d if len(w) > 1]  #길이가 1인 토큰은 제외
    POS_description_noun_join.append(" ".join(d2))  # 토큰을 연결(join)하여 리스트 구성
print(POS_description_noun_join)  #작업 확인용 출력
NEG_description = NEG_data_df['description']

NEG_description_noun_tk = []
NEG_description_noun_join = []

for d in NEG_description:
    NEG_description_noun_tk.append(okt.nouns(d))  #형태소가 명사인 것만 추출

for d in NEG_description_noun_tk:
    d2 = [w for w in d if len(w) > 1]  #길이가 1인 토큰은 제외
    NEG_description_noun_join.append(" ".join(d2))  # 토큰을 연결(join)하여 리스트 구성
print(NEG_description_noun_join)  #작업 확인용 출력
POS_tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, min_df=2)
POS_dtm = POS_tfidf.fit_transform(POS_description_noun_join)
POS_vocab = dict()

for idx, word in enumerate(POS_tfidf.get_feature_names_out()):
    POS_vocab[word] = POS_dtm.getcol(idx).sum()

POS_words = sorted(POS_vocab.items(), key=lambda x: x[1], reverse=True)
POS_words  #작업 확인용 출력
NEG_tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, min_df=2)
NEG_dtm = NEG_tfidf.fit_transform(NEG_description_noun_join)
NEG_vocab = dict()

for idx, word in enumerate(NEG_tfidf.get_feature_names_out()):
    NEG_vocab[word] = NEG_dtm.getcol(idx).sum()

NEG_words = sorted(NEG_vocab.items(), key=lambda x: x[1], reverse=True)
NEG_words  #작업 확인용 출력
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

fm._get_fontconfig_fonts()
font_location = 'C:/Windows/Fonts/malgun.ttf'
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

max = 15  #바 차트에 나타낼 단어의 수
plt.bar(range(max), [i[1] for i in POS_words[:max]], color="blue")
plt.title("긍정 뉴스의 단어 상위 %d개" % max, fontsize=15)
plt.xlabel("단어", fontsize=12)
plt.ylabel("TF-IDF의 합", fontsize=12)
plt.xticks(range(max), [i[0] for i in POS_words[:max]], rotation=70)

plt.show()
plt.bar(range(max), [i[1] for i in NEG_words[:max]], color="red")
plt.title("부정 뉴스의 단어 상위 %d개" % max, fontsize=15)
plt.xlabel("단어", fontsize=12)
plt.ylabel("TF-IDF의 합", fontsize=12)
plt.xticks(range(max), [i[0] for i in NEG_words[:max]], rotation=70)

plt.show()

In [ ]:
import gensim
import gensim.corpora as corpora

description = data_df['description']
description_noun_tk = []

for d in description:
    description_noun_tk.append(okt.nouns(d))  #형태소가 명사인 것만 추출

description_noun_tk2 = []

for d in description_noun_tk:
    item = [i for i in d if len(i) > 1]  #토큰의 길이가 1보다 큰 것만 추출
    description_noun_tk2.append(item)
dictionary = corpora.Dictionary(description_noun_tk2)
print(dictionary[1])  #작업 확인용 출력
corpus = [dictionary.doc2bow(word) for word in description_noun_tk2]
print(corpus)  #작업 확인용 출력
# ✔️ LDA는 문서에서 숨겨진 토픽을 찾아내는 알고리즘
# ✔️ 토픽을 정의할 필요 없이, 문서에서 자동으로 토픽을 추출
# ✔️ 토픽마다 특정 단어들이 높은 확률로 등장
# ✔️ 각 문서가 어떤 토픽에 속하는지 분석 가능
k = 4  #토픽의 개수 설정
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus, iterations=12, num_topics=k, id2word=dictionary, passes=1,
                                                    workers=10)
# corpus: 문서를 토큰화하여 BOW(Bag-of-Words) 형식으로 변환한 데이터
# iterations=12: 각 문서에서 토픽을 업데이트하는 반복 횟수
# num_topics=k: 찾을 토픽 개수 (여기서는 4개)
# id2word=dictionary: 단어 ID와 단어 매핑 정보
# passes=1: 전체 데이터셋을 몇 번 학습할지 (값이 클수록 학습이 더 많이 됨)
# workers=10: 병렬처리를 위한 CPU 코어 개수

print(lda_model.print_topics(num_topics=k, num_words=15))
#최초 한번만 설치
# !pip install pyLDAvis
# !pip install --upgrade pandas
import pickle

with open("lda_model.pkl", "wb") as f:
    pickle.dump(lda_model, f)

import pyLDAvis.gensim

lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
# lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, n_jobs=1)

In [ ]:
pyLDAvis.display(lda_vis)
